In [1]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
#from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import numpy as np
import spacy 
import urllib
import string

In [2]:
#loads the english module of spacy 
nlp = spacy.load('en')

In [3]:
#opens the war and peace text
Google_AI_raw = open('./input/nyt_google_ai.txt').read()

In [4]:
#tokenizes each item in nyt article
Google_AI_lower = Google_AI_raw.lower()
Google_AI = nlp(Google_AI_lower)

In [5]:
from collections import Counter

In [6]:
top_words=Counter([entry.string.strip() for entry in Google_AI
                  if entry.is_alpha and not entry.is_stop]).most_common(10000)

In [7]:
just_words = [item[0] for item in top_words]

In [8]:
Google_ai_sentences = LineSentence('./input/nyt_google_ai.txt')

In [9]:
model = Word2Vec(Google_ai_sentences, size=50, window=5, min_count=3, seed=2, workers=4, iter=10)

In [10]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file 
top_words_nyt=[]
for entry in just_words:
    if entry in model.wv.vocab:
        top_words_nyt.append(entry)

In [11]:
words_nyt_model_vec = model[top_words_nyt]

In [12]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_tsne = tsne.fit_transform(words_nyt_model_vec)

In [13]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_tsne[:,0],
                                    x2=words_tsne[:,1],
                                    names=top_words_nyt))
p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)